In [24]:
import os
import pandas as pd
import numpy as np
import re
import geopandas as gpd
import zipfile as zf
import shapefile
from shapely.geometry import shape
from io import StringIO
from osgeo import osr
import matplotlib.pyplot as plt

In [10]:
path = re.sub(r'\\Experimental Notebooks$', '', os.getcwd())
path = path + '\\Data'
data_files = [path + '\\' +  x for x in os.listdir(path)]

Trying to read the shape files

In [20]:
zip_path = data_files[1]
zp = zf.ZipFile(zip_path)
files_to_read = [y for y in zp.namelist() for ending in ['dbf', 'prj', 'shp', 'shx'] if y.endswith(ending)]
dummy = zp.read(files_to_read[0])
dummy.__class__
dbf_file, shp_file, shx_file, prj_file = [zp.open(filename) for filename in files_to_read]

r = shapefile.Reader(shp = shp_file, shx = shx_file, dbf = dbf_file)
r.numRecords
r.__class__
r.fields[3:]
attributes, geometry = [], []
field_names = [field[0] for field in r.fields[1:]]
for row in r.shapeRecords():
    geometry.append(shape(row.shape.__geo_interface__))
    attributes.append(dict(zip(field_names, row.record)))

#prj_file.read()
#osr.SpatialReference(prj_file.read())
#proj4_string = osr.SpatialReference(prj_file).ExportToProj4()
gdf = gpd.GeoDataFrame(data = attributes, geometry = geometry)

In [23]:
gdf.head()
gdf.columns

Index(['ISOURBID', 'ISO3', 'URBID', 'NAME', 'SCHNM', 'ES90POP', 'ES95POP',
       'ES00POP', 'SQKM_FINAL', 'URB_D_MEAN', 'BUF_D_MEAN', 'D_T_DIFF',
       'URB_N_MEAN', 'BUF_N_MEAN', 'N_T_DIFF', 'LATITUDE', 'LONGITUDE',
       'geometry'],
      dtype='object')

In [31]:
disasters_df = pd.read_excel(data_files[0], skiprows = 6)
disasters_df.columns


Index(['Dis No', 'Year', 'Seq', 'Disaster Group', 'Disaster Subgroup',
       'Disaster Type', 'Disaster Subtype', 'Disaster Subsubtype',
       'Event Name', 'Entry Criteria', 'Country', 'ISO', 'Region', 'Continent',
       'Location', 'Origin', 'Associated Dis', 'Associated Dis2',
       'OFDA Response', 'Appeal', 'Declaration', 'Aid Contribution',
       'Dis Mag Value', 'Dis Mag Scale', 'Latitude', 'Longitude', 'Local Time',
       'River Basin', 'Start Year', 'Start Month', 'Start Day', 'End Year',
       'End Month', 'End Day', 'Total Deaths', 'No Injured', 'No Affected',
       'No Homeless', 'Total Affected', 'Reconstruction Costs ('000 US$)',
       'Insured Damages ('000 US$)', 'Total Damages ('000 US$)', 'CPI'],
      dtype='object')

In [33]:
print(set(disasters_df['Disaster Type']))
print(set(disasters_df['Disaster Subtype']))

{'Epidemic', 'Drought', 'Wildfire', 'Extreme temperature '}
{nan, 'Drought', 'Forest fire', 'Land fire (Brush, Bush, Pasture)', 'Viral disease', 'Parasitic disease', 'Heat wave', 'Bacterial disease'}


In [46]:
disasters_df.filter(regex = 'Year|Disaster Type|Disaster Subtype').pivot_table(index = ['Year', 'Disaster Type', 'Disaster Subtype'], aggfunc = 'count')

Disaster Subtype  End Year  Start Year
Year Disaster Type                                               
1900 Drought                              2         2           2
     Epidemic                             1         1           1
1901 Epidemic                             0         1           1
1903 Drought                              1         1           1
1906 Drought                              1         1           1
...                                     ...       ...         ...
2020 Epidemic                             6         6           6
     Extreme temperature                  4         4           4
     Wildfire                             5         8           8
2021 Drought                              1         1           1
     Wildfire                             2         2           2

[269 rows x 3 columns]

So, Information we have.

- Year of outbreaks of diseases, wildfires, droughts and heatwaves.
- Geo-referenced files with urban parts of the planet. 
- Urban extent in km 
- Urban land surface average temperature 
Documentation on the variables inside the geographical Pandas Object is here:
https://sedac.ciesin.columbia.edu/data/set/sdei-global-uhi-2013/docs